In [ ]:
# create the master line list

import os

import lsst.daf.persistence as dafPersist
from lsst.obs.pfs.constructLineList import ConstructLineListTask
from lsst.utils import getPackageDir
import pfs.drp.stella as drpStella

%matplotlib inline
#%matplotlib notebook

path = "/Volumes/external/azuri/spectra/pfs/LAM"

# create dataId for Arc
arcVisits = [2114,2579,2580,2581,2118]
spectrograph = 1
arm = 'r'

# create butler
butler = dafPersist.Butler(path)
print 'butler created'

# create butler subset
dataRefList = []
dataIdArc = dict(field="ARC", spectrograph=spectrograph, arm=arm)
for visit in arcVisits:
    dataIdArc['visit'] = visit
    dataRefListTemp = [ref for ref in butler.subset("postISRCCD", 'visit', dataIdArc)]
    dataRefList.append(dataRefListTemp[0])
print 'len(dataRefList) = ',len(dataRefList)

# create config
config = ConstructLineListTask.ConfigClass()

# apply config overrides
ConstructLineListTask.applyOverrides(config)
constructLineListTask = ConstructLineListTask(config=config)
constructLineListTask.config.plot = [7,68,196,258,315,337,394,456,583,645]
constructLineListTask.config.aperturesToExtract = [-1]
constructLineListTask.config.fwhm = 2.2
constructLineListTask.config.order = 5
constructLineListTask.config.clobber = True

# set logLevel
logLevel = {0: "WARN", 1: "INFO", 2: "DEBUG", 3: "TRACE"}[0]
constructLineListTask.logLevel = logLevel

# create master line list
# This will print an 'OptimizeWarning' which you can safely ignore
constructLineListTask.run(dataRefList, butler)

print 'done'

In [ ]:
from pfs.drp.stella.reduceArcTask import ReduceArcTask
from pfs.drp.stella.utils import plotWavelengthResiduals
import lsst.log as log

myReduceArcTask = ReduceArcTask()

wLenFile = os.path.join(getPackageDir('obs_pfs'), 'pfs/RedFiberPixels.fits.gz')

# set reduceArcTask parameters and run
myReduceArcTask.config.plot = 2
myReduceArcTask.logLevel=logLevel

dataRefList = []
dataIdArc = dict(field="ARC", spectrograph=spectrograph, arm=arm)
elements = ['Ne','Hg,Ar','Hg,Ar','Hg,Ar','Ne,Hg,Ar']
for iVisit in range(len(arcVisits)):
    visit = arcVisits[iVisit]
    element = elements[iVisit]
    print 'visit = ',visit,', element = ',element
    dataIdArc['visit'] = visit
    dataRefList = [ref for ref in butler.subset("postISRCCD", 'visit', dataIdArc)]
    myReduceArcTask.config.elements=element

    spectrumSetFromProfile, measuredLinesPerArc, offsetPerArc, flatFiberTraceSet = myReduceArcTask.run(dataRefList, butler, wLenFile)

    print 'type(spectrumSetFromProfile) = ',type(spectrumSetFromProfile)

    for iTrace in range(spectrumSetFromProfile.size()):
        print 'fiberTraceId[',iTrace,'] = ',spectrumSetFromProfile.getSpectrum(iTrace).getITrace()
    
    for fiberTraceId in [7,68,196,258,315,337,394,456,583,645]:
        fiberIdx = int(flatFiberTraceSet.getIndexWithId(fiberTraceId))
        lineList = measuredLinesPerArc[0][fiberIdx]
        print 'type(fiberIdx) = ',type(fiberIdx)

        logger = log.Log.getLogger("plotWavelengthResiduals")
        logger.setLevel(log.TRACE)
        logger = log.Log.getLogger("reduceArcTask")
        logger.setLevel(log.INFO)
        spectrum = spectrumSetFromProfile.getSpectrum(fiberIdx)
        plotWavelengthResiduals(lineList, fiberTraceId, spectrum.getDispRms())    